In [ ]:
# 가져다 쓸 모듈들
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # 예쁜 matplotlib
import folium # 지도 시각화
import json # geojson 파일용
import geopandas as gpd # geojson 파일용
import matplotlib.font_manager as fm # 한글 폰트 사용
import math # 제곱근
import scipy # 상관 계수
import statsmodels.api as sm # 다중회귀
import statsmodels.formula.api as smf # 다중회귀
from sklearn.preprocessing import StandardScaler # 표준화를 위해 필요
from sklearn.model_selection import train_test_split # 학습용/테스트 데이터

fm.get_fontconfig_fonts()
font_location = "malgun.ttf"
font_name = fm.FontProperties(fname=font_location).get_name()
plt.rc('font', family=font_name)

# pd.set_option('display.max_row', 30) # 더 많이 보고 싶을때

In [ ]:
filename = "3.대전광역시_신호등(보행등).geojson"
file = open(filename)
df3 = gpd.read_file(file)
plz=(df3.head(10))
plz
gc_plz=df3[df3['dong']=='판암동'].reset_index()
len(gc_plz)
gc_plz

In [ ]:
converted_json = gc_plz.to_json()

m = folium.Map(
    location=[36.34156,127.39704],
    zoom_start=11
)

folium.GeoJson(
    converted_json,
).add_to(m)

m

In [ ]:
plz

In [ ]:
filename = "2.대전광역시_교통사고격자(2017~2019).geojson"
file = open(filename)
df2 = gpd.read_file(file)
gdf=df2[df2['acci_cnt']>=30]
gdf=gdf.reset_index()
gdf.head(10)
gdf2=gdf.copy()

In [ ]:
print(df2['geometry'].iloc[[5636,12176,8948]])
temp=df2['geometry'].iloc[[5636,12176,8948]]
converted_json = temp.to_json()

m = folium.Map(
    location=[36.34156,127.39704],
    zoom_start=11
)

folium.GeoJson(
    converted_json,
).add_to(m)

m

In [ ]:
for i in plz['geometry']:
    for j in df2['geometry']:
        if i.within(j)==True:
            print(j)

In [ ]:
gc=[]
for i in range(len(gc_plz['geometry'])):
    for j in range(len(df2['geometry'])):
        if gc_plz['geometry'][i].within(df2['geometry'][j])==True:
            print(j)
            gc.append(j)
print(df2['geometry'].iloc[gc])
gc_2=df2['geometry'].iloc[gc]

converted_json = gc_2.to_json()

m = folium.Map(
    location=[36.34156,127.39704],
    zoom_start=11
)

folium.GeoJson(
    converted_json,
).add_to(m)

m

In [ ]:
plz['geometry'][2]

In [ ]:
len(df2)

In [ ]:
gc=[]
for i in df2['geometry']:
    for j in plz['geometry']:
        if i.contains(j)==True:
            gc.append(i)

In [ ]:
df2.iloc[[10,20]]

In [ ]:
for i in gdf2['geometry']:
    print(i.area)

In [ ]:
from pyproj import Proj, transform
import shapely.geometry as geom
import shapely.wkt

inProj = Proj({'init': 'epsg:4326'})
outProj = Proj({'init': 'epsg:4326'})

for i in range(len(gdf.geometry)):
    region = gdf.geometry[i]
    pols = list(region)
    for j in range(len(pols)) :
        coords_obj = list(pols[j].exterior.coords)
        empty_list = []            
        for k in range(len(coords_obj)):
            x,y = transform(inProj,outProj,coords_obj[k][0],coords_obj[k][1])
            empty_list.append(geom.Point(x,y))
        empty_list = geom.Polygon([p.x,p.y] for p in empty_list)
        pols[j] = empty_list
    result = geom.MultiPolygon(pols)
    gdf2.geometry[i] = result

In [ ]:
ga=df2.copy()
df2

In [ ]:
for i in range(len(gdf.geometry)) :
    region = gdf.geometry[i]
    if region.type == "Polygon" :
        coords_obj = list(region.exterior.coords)
        empty_list = []
        for j in range(len(coords_obj)) :
            x,y = transform(inProj,outProj,coords_obj[j][0],coords_obj[j][1])
            empty_list.append(geom.Point(x,y))
        result = geom.Polygon([[p.x,p.y] for p in empty_list])
    elif region.type == "MultiPolygon" :
        pols = list(region)
        for j in range(len(pols)) :
            coords_obj = list(pols[j].exterior.coords)
            empty_list = []            
            for k in range(len(coords_obj)):
                x,y = transform(inProj,outProj,coords_obj[k][0],coords_obj[k][1])
                empty_list.append(geom.Point(x,y))
            empty_list = geom.Polygon([p.x,p.y] for p in empty_list)
            print(empty_list)
            pols[j] = empty_list
        result = geom.MultiPolygon(pols)
        #print(result)
    ga.geometry[i] = result
    #print(gdf2.geometry[i])
print("Done")
ga.head(50)

In [ ]:
import numpy as np
a = np.stack([df2['geometry'].centroid.x, df2['geometry'].centroid.y], axis=1)#.reshape([1, -1, 2])
a